In [1]:
%autosave 0

Autosave disabled


In [2]:
# !pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

In [3]:
# !pip install keras-image-helper

In [4]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [5]:
from keras_image_helper import create_preprocessor

In [6]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [7]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [8]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [9]:
# !wget http://bit.ly/mlbookcamp-pants -O pants.jpg

In [10]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [11]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [12]:
ls

 Volume in drive C has no label.
 Volume Serial Number is BCC0-49D8

 Directory of C:\Users\KonuTech\ml-zoomcamp\machine-learning-zoomcamp\10-kubernetes\code

03.12.2023  13:16    <DIR>          .
03.12.2023  13:16    <DIR>          ..
02.12.2023  11:40    <DIR>          .ipynb_checkpoints
03.12.2023  13:13    <DIR>          clothing-model
07.12.2021  13:41        86˙185˙888 clothing-model.h5
17.09.2023  17:17               234 docker-compose.yaml
17.09.2023  17:17             1˙807 gateway.py
17.09.2023  17:17               268 image-gateway.dockerfile
17.09.2023  17:17               110 image-model.dockerfile
17.09.2023  17:17    <DIR>          kube-config
03.12.2023  12:57            23˙048 pants.jpg
17.09.2023  17:17    <DIR>          ping
17.09.2023  17:17               258 Pipfile
17.09.2023  17:17            23˙839 Pipfile.lock
17.09.2023  17:17             2˙110 plan.md
17.09.2023  17:17               785 proto.py
17.09.2023  17:17             2˙009 README.md
17.09.2023  17:17 

In [13]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [14]:
preds = pb_response.outputs['dense_7'].float_val

In [15]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [16]:
dict(zip(classes, preds))

{'dress': -1.8798640966415405,
 'hat': -4.756312370300293,
 'longsleeve': -2.3595328330993652,
 'outwear': -1.0892646312713623,
 'pants': 9.90378475189209,
 'shirt': -2.826181173324585,
 'shoes': -3.6483113765716553,
 'shorts': 3.2411553859710693,
 'skirt': -2.612095355987549,
 't-shirt': -4.852035999298096}